Paper 
# Active clustering

# Import the required library

In [ ]:
import numpy as np

# Load a XES file and exploration

In [1]:
from pm4py.objects.log.importer.xes import factory as xes_import_factory
log = xes_import_factory.apply('PrepaidTravelCost.xes')

print(
    'length of trace', len(log),
    '\nlength of event', sum(len(trace) for trace in log)
)

/home/bk/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecatedWarning: apply is deprecated as of 1.3.0 and will be removed in 2.0.0. Use importer entrypoint instead
  


length of trace 2099 
length of event 18246


# Filtering out some variatns 

In [2]:
from pm4py.algo.filtering.log.variants import variants_filter
variants = variants_filter.get_variants(log)

from pm4py.statistics.traces.log import case_statistics
variants_count = case_statistics.get_variant_statistics(log)
variants_count = sorted(variants_count, key=lambda x: x['count'], reverse=True)

In [3]:
#parameter
w = 0.5  #window size
tf = 0.8 #target fitness
nb_clus = 3 #number of cluster
N = 1 #case separate 




# Adding statistics about Frequency and Acc. Frequency 

In [6]:
def add_frequency_into_variants_count(variants_count) : 
    #total sum
    s = 0 
    for i in range(len(variants_count)) : 
        s += variants_count[i]['count'] 

    #frequency 
    for i in range(len(variants_count)) : 
        variants_count[i]['freq']  = variants_count[i]['count'] / s
        if i == 0 : 
            variants_count[i]['acc_freq']  = variants_count[i]['freq']
        else : 
            variants_count[i]['acc_freq'] = variants_count[i-1]['acc_freq'] + variants_count[i]['freq']
    return variants_count

In [7]:
#GL, grouped event log = A collection of dpi. Used the same notation in the paper.
GL = add_frequency_into_variants_count(variants_count)

In [ ]:
variants_count[:5]

In [ ]:
def filtering_with_freq(variants_count, w) :
    acc_freq = 0 
    last_idx = 0
    while (variants_count[last_idx]['acc_freq']  <= w) : 
        last_idx += 1
    W = variants_filter.apply(
        log, 
        [variants_count[i]['variant'] for i in range(last_idx+1)]  
    )
    W_variant = case_statistics.get_variant_statistics(W)
    W_variant = sorted(W_variant, key=lambda x: x['count'], reverse=True)
    W_variant = add_frequency_into_variants_count(W_variant)
    return W, W_variant

In [ ]:
W, W_variant = filtering_with_freq(variants_count, 0.5)

# Line 4 $$while(|CS| < nb_{clus}) \wedge (R \neq \emptyset) do $$

In [114]:
#Corresponding to line 5,6
C=[]
I=[]

# HERUSTIC MINER

In [109]:
# corresponding to the line 3 in the paper. 
R=[]
for gl in GL: 
    R.append(gl['variant'])
    
R_variants_count = GL

# R_variants_count contains 
#     variant : trace
#     count : number of occurence
#     freq : frequency
#     acc_freq : accmulated freq.


In [96]:
# line 10
cur_dpi = R_variants_count[0]['variant'] #since R has been already sorted in order of increase in frequency

In [97]:
cur_dpi

'Permit SUBMITTED by EMPLOYEE,Permit APPROVED by ADMINISTRATION,Permit FINAL_APPROVED by SUPERVISOR,Request For Payment SUBMITTED by EMPLOYEE,Request For Payment APPROVED by ADMINISTRATION,Request For Payment FINAL_APPROVED by SUPERVISOR,Request Payment,Payment Handled'

# Line 15, PM<-HM(C U {cur_dpi})

In [99]:
R.remove(cur_dpi)

In [111]:
try : 
    print(R.index(cur_dpi))
    
except :
    print("cur_dpi is not exist")

0


In [112]:
R[0]

'Permit SUBMITTED by EMPLOYEE,Permit APPROVED by ADMINISTRATION,Permit FINAL_APPROVED by SUPERVISOR,Request For Payment SUBMITTED by EMPLOYEE,Request For Payment APPROVED by ADMINISTRATION,Request For Payment FINAL_APPROVED by SUPERVISOR,Request Payment,Payment Handled'

In [123]:
C=[]
cur_dpi = R[0]

In [124]:
cur_dpi

'Permit SUBMITTED by EMPLOYEE,Permit APPROVED by ADMINISTRATION,Permit FINAL_APPROVED by SUPERVISOR,Request For Payment SUBMITTED by EMPLOYEE,Request For Payment APPROVED by ADMINISTRATION,Request For Payment FINAL_APPROVED by SUPERVISOR,Request Payment,Payment Handled'

In [127]:
len(C + [cur_dpi] + [cur_dpi])

2

In [137]:
log

[{'attributes': {'Rfp_id': 'request for payment 73550', 'Permit travel permit number': 'UNKNOWN', 'Task': 'task 71977', 'OrganizationalEntity': 'organizational unit 65463', 'RequestedAmount': 854.5798378437331, 'Activity': 'activity 505', 'Permit TaskNumber': 'UNKNOWN', 'Permit BudgetNumber': 'budget 6198', 'Permit ProjectNumber': 'UNKNOWN', 'Project': 'project 503', 'concept:name': 'request for payment 73550', 'Permit OrganizationalEntity': 'organizational unit 65455', 'Permit RequestedBudget': 1979.272103997747, 'Cost Type': 0, 'Permit id': 'travel permit 73549', 'Permit ActivityNumber': 'UNKNOWN', 'RfpNumber': 'request for payment number 73551'}, 'events': [{'id': 'st_step 73555_0', 'org:resource': 'STAFF MEMBER', 'concept:name': 'Permit SUBMITTED by EMPLOYEE', 'time:timestamp': datetime.datetime(2017, 1, 9, 14, 48, 43, tzinfo=datetime.timezone(datetime.timedelta(seconds=3600))), 'org:role': 'EMPLOYEE'}, '..', {'id': 'st_step 73556_0', 'org:resource': 'STAFF MEMBER', 'concept:name':

In [169]:
C=[]

In [188]:
from pm4py.algo.discovery.heuristics import factory as heuristics_miner
from pm4py.evaluation.replay_fitness import factory as replay_factory

def fit_check_w_HM(log :list, cur_dpi :list, C :list, tf :float) -> (list, list): 
    log = variants_filter.apply(
            log, 
            [c for c in C + [cur_dpi]]  
    )
    heu_net = heuristics_miner.apply_heu(log, parameters={"dependency_thresh": 0.99})
    net, im, fm = heuristics_miner.apply(log, parameters={"dependency_thresh": 0.99})
    fit = replay_factory.apply(C_log, net, im, fm )
    print("fitness {}".format(fit['averageFitness']))
    if fit['averageFitness'] > tf :
        C.append(cur_dpi)
        C = list(set(C))
        try : 
            R.remove(cur_dpi)
            
        except : 
            print("not found")
            print
            pass
    
    else : 
        pass
        # if cluster is larger enough
        # then : look ahead
        # else : I <- I U {cur_dpi}
        
    
    return C, R

In [189]:
C,R = fit_check_w_HM(log, cur_dpi, C, tf)

/home/bk/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: DeprecatedWarning: apply_heu is deprecated as of 1.3.0 and will be removed in 2.0.0. Use algorithm entrypoint instead
  if __name__ == '__main__':
/home/bk/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: DeprecatedWarning: apply is deprecated as of 1.3.0 and will be removed in 2.0.0. Use algorithm entrypoint instead
  # Remove the CWD from sys.path while we load stuff.
/home/bk/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecatedWarning: apply is deprecated as of 1.3.0 and will be removed in 2.0.0. Use evaluator entrypoint instead
  # This is added back by InteractiveShellApp.init_path()


fitness 0.8000000000000076
not found


In [187]:
C = R[0:100]

In [180]:
R

['Permit SUBMITTED by EMPLOYEE,Permit APPROVED by ADMINISTRATION,Permit APPROVED by BUDGET OWNER,Permit FINAL_APPROVED by SUPERVISOR,Request For Payment SUBMITTED by EMPLOYEE,Request For Payment APPROVED by ADMINISTRATION,Request For Payment APPROVED by BUDGET OWNER,Request For Payment FINAL_APPROVED by SUPERVISOR,Request Payment,Payment Handled',
 'Permit SUBMITTED by EMPLOYEE,Permit APPROVED by ADMINISTRATION,Permit APPROVED by SUPERVISOR,Permit FINAL_APPROVED by DIRECTOR,Request For Payment SUBMITTED by EMPLOYEE,Request For Payment APPROVED by ADMINISTRATION,Request For Payment FINAL_APPROVED by SUPERVISOR,Request Payment,Payment Handled',
 'Permit SUBMITTED by EMPLOYEE,Permit APPROVED by ADMINISTRATION,Permit APPROVED by BUDGET OWNER,Permit APPROVED by SUPERVISOR,Permit FINAL_APPROVED by DIRECTOR,Request For Payment SUBMITTED by EMPLOYEE,Request For Payment APPROVED by ADMINISTRATION,Request For Payment APPROVED by BUDGET OWNER,Request For Payment FINAL_APPROVED by SUPERVISOR,Reque

# line 16 fitness check 

In [ ]:
variants_count[0]['variant'].split(',')

In [ ]:
def dist_btw_traces(trace1,trace2, output=False) : 
    A = set(trace1.split(','))
    B = set(trace2.split(','))
    if output : 
        print("\nA SET", A)
        print("\nB SET", B)
        print("\nUNION", A.union(B))
        print("\nINTERSECTION", A.intersection(B))
        print("\nDIFFERENCE", A.union(B) - A.intersection(B))
    return(len(A.union(B) - A.intersection(B)))

In [ ]:
dist_btw_traces(variants_count[0]['variant'], variants_count[1]['variant'], True)

In [ ]:
C = 

In [ ]:
W_variant

In [ ]:
def min_distance_seeker(dpi, C) :
    import sys
    min_dist = sys.maxsize #Instead of inf. 
    for c in C : 
        if min_dist > dist_btw_traces(dpi,c) : 
            min_dpi = c
    return min_dpi

In [ ]:
min_distance_seeker(
    variants_count[0]['variant'],
    

)

In [ ]:
#Initialization
R = [i for i in range(len(variants_count))]

In [ ]:
for r in R :
    dist_list = []
    dist_dpi = 0 #temp variable for distance sum
    min_dpi_dist = np.inf
    min_dpi = np.NaN
    
    for c in C : 
        dist_dpi += distance_of_two_set(r,c, variants_count)
    if dist_dpi / len(C) < min_dpi_dist : #if average is less than min_dpi, then update
        min_dpi_dist = dist_dpi / len(C)
        min_dpi = r
        
    print("among R variant", variants_count[r]['variant'], min_dpi_dist/len(C))

fitness_inductive= 1.0


/home/bk/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecatedWarning: apply is deprecated as of 1.3.0 and will be removed in 2.0.0. Use evaluator entrypoint instead
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
distance_of_two_set(1,9,variants_count, True)

In [ ]:
#parameter
w = 0.5  #window size
tf = 0.8 #target fitness
nb_clus = 3 #number of cluster
N = 1 #case separate 




from pm4py.visualization.heuristics_net import factory as hn_vis_factory
gviz = hn_vis_factory.apply(heu_net)
hn_vis_factory.view(gviz)

from pm4py.visualization.petrinet import factory as pn_vis_factory
gviz = pn_vis_factory.apply(net, im, fm)
pn_vis_factory.view(gviz)